In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import os
import ast
import nltk
import pickle
import math

In [2]:
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectPercentile, chi2, SelectKBest, f_classif, mutual_info_classif

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
TEST_SIZE = 0.2
RANDOM_STATE = 42
SAMPLE_SIZE = 3000

TEXT_COL_NAME = 'text_split'
TEXT_TOKEN_COL_NAME = 'text_token_stop'
CLASS_COL_NAME = 'reduced_tags'

# DF_FILE_PATH = './data/no_stop.pkl'
DF_FILE_PATH = './data/no_stop_1000_30.pkl'

In [5]:
# DT_SAVE_FILE_NAME = '/content/drive/MyDrive/資料探勘/project/Model/DT_1000.bin'
# DT_SELECT_SAVE_FILE_NAME = '/content/drive/MyDrive/資料探勘/project/Model/DT_selection.bin'
# RF_SAVE_FILE_NAME = '/content/drive/MyDrive/資料探勘/project/Model/RF_1000.bin'
# NN_SAVE_FILE_NAME = '/content/drive/MyDrive/資料探勘/project/Model/NN_model'

# Load Data

In [6]:
%%time
# df = pd.read_csv('./data/medium_articles.csv')
df = pd.read_pickle(DF_FILE_PATH)
# print(final_df.shape)

df.head()

CPU times: user 544 ms, sys: 131 ms, total: 675 ms
Wall time: 696 ms


,title,text,url,authors,timestamp,tags,text_length,reduced_tags,text_token,text_token_stop,text_after
0,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology...",2326,[Health],"[You’ve, heard, of, him,, haven’t, you?, Phine...","[You’ve, heard, him,, haven’t, you?, Phineas, ...",234
1,Quora Overview,Making the most of Quora for content marketing...,https://medium.com/digital-marketing-lab/quora...,['Casey Botticello'],2020-09-04 18:30:41.246000+00:00,"['Productivity', 'Entrepreneurship', 'Writing'...",1887,"[Entrepreneurship, Writing, Startup]","[Making, the, most, of, Quora, for, content, m...","[Making, Quora, content, marketing, There, sev...",193
2,The Simple Formula For Becoming A Better Writer,You Need To Show Up Every Single Day. “The mos...,https://medium.com/swlh/the-simple-formula-for...,['Matt Lillywhite'],2020-06-26 13:31:34.246000+00:00,"['Creativity', 'Entrepreneurship', 'Blogging',...",1708,"[Entrepreneurship, Writing]","[You, Need, To, Show, Up, Every, Single, Day.,...","[You, Need, To, Show, Up, Every, Single, Day.,...",195
3,The Power of Sleep in Learning: Mind-Blowing S...,The Power of Sleep in Learning: Mind-Blowing S...,https://medium.com/superintelligence/the-power...,['John Von Neumann Ii'],2020-02-12 09:19:16.941000+00:00,"['Self Improvement', 'Productivity', 'Science'...",2201,"[Self Improvement, Health, Entrepreneurship]","[The, Power, of, Sleep, in, Learning:, Mind-Bl...","[The, Power, Sleep, Learning:, Mind-Blowing, S...",231
4,All the Love you do not see,All the Love you do not see On writing with he...,https://asingularstory.medium.com/all-the-love...,['A Singular Story'],2020-06-13 12:29:45.505000+00:00,"['Social Media', 'Future', 'Society', 'Writing...",1396,"[Writing, Mental Health]","[All, the, Love, you, do, not, see, On, writin...","[All, Love, see, On, writing, heart, hope, Pho...",150


In [7]:
df.head()

,title,text,url,authors,timestamp,tags,text_length,reduced_tags,text_token,text_token_stop,text_after
0,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology...",2326,[Health],"[You’ve, heard, of, him,, haven’t, you?, Phine...","[You’ve, heard, him,, haven’t, you?, Phineas, ...",234
1,Quora Overview,Making the most of Quora for content marketing...,https://medium.com/digital-marketing-lab/quora...,['Casey Botticello'],2020-09-04 18:30:41.246000+00:00,"['Productivity', 'Entrepreneurship', 'Writing'...",1887,"[Entrepreneurship, Writing, Startup]","[Making, the, most, of, Quora, for, content, m...","[Making, Quora, content, marketing, There, sev...",193
2,The Simple Formula For Becoming A Better Writer,You Need To Show Up Every Single Day. “The mos...,https://medium.com/swlh/the-simple-formula-for...,['Matt Lillywhite'],2020-06-26 13:31:34.246000+00:00,"['Creativity', 'Entrepreneurship', 'Blogging',...",1708,"[Entrepreneurship, Writing]","[You, Need, To, Show, Up, Every, Single, Day.,...","[You, Need, To, Show, Up, Every, Single, Day.,...",195
3,The Power of Sleep in Learning: Mind-Blowing S...,The Power of Sleep in Learning: Mind-Blowing S...,https://medium.com/superintelligence/the-power...,['John Von Neumann Ii'],2020-02-12 09:19:16.941000+00:00,"['Self Improvement', 'Productivity', 'Science'...",2201,"[Self Improvement, Health, Entrepreneurship]","[The, Power, of, Sleep, in, Learning:, Mind-Bl...","[The, Power, Sleep, Learning:, Mind-Blowing, S...",231
4,All the Love you do not see,All the Love you do not see On writing with he...,https://asingularstory.medium.com/all-the-love...,['A Singular Story'],2020-06-13 12:29:45.505000+00:00,"['Social Media', 'Future', 'Society', 'Writing...",1396,"[Writing, Mental Health]","[All, the, Love, you, do, not, see, On, writin...","[All, Love, see, On, writing, heart, hope, Pho...",150


In [8]:
print(df.shape)
# (192368, 6)
# (119778, 8)

(18118, 11)


In [9]:
# df = df[:SAMPLE_SIZE]

In [10]:
%%time
df['text'] = df['text'].apply(lambda x: x.replace('\n\n', ' '))
df['text_token'] = df['text'].apply(lambda x: x.split(' '))
df['text_token']

CPU times: user 303 ms, sys: 81.6 ms, total: 385 ms
Wall time: 385 ms


0        [You’ve, heard, of, him,, haven’t, you?, Phine...
1        [Making, the, most, of, Quora, for, content, m...
2        [You, Need, To, Show, Up, Every, Single, Day.,...
3        [The, Power, of, Sleep, in, Learning:, Mind-Bl...
4        [All, the, Love, you, do, not, see, On, writin...
                               ...                        
18113    [My, theory, on, the, future, of, customer, se...
18114    [I, recently, downloaded, an, app, of, a, comp...
18115    [In, case, you, don’t, know,, now, you, can, u...
18116    [25, cent, chicken, wings., That’s, how, it, a...
18117    [Photo, by, Comeup, 2021, on, Comeup, 2021, Th...
Name: text_token, Length: 18118, dtype: object

In [11]:
df['text'][0]

"You’ve heard of him, haven’t you? Phineas Gage. The railroad worker who survived an explosion that involved an iron rod piercing through his left cheek and out of his brain and skull. Yeah. I know. You’re probably wondering “yeah, alright sweet. What about him?” Well, let’s just say that he was a really popular patient for the field of neuroscience (Cherry, par. 1). And what I found the most interesting about this tragic event was the science of his behavior afterward. For those of you who don’t know much about Phineas Gage, let me fill you in with the help of my research. Phineas Gage, 25 years old, was a railroad worker in Vermont. One day, at work, he was using an iron rod to handle explosive gun powder. As he was using the iron rod to handle the gun powder, an explosion suddenly occurred. The iron rod then went through his left cheek and brain. Fortunately, he survived and was able to talk and walk after the accident (Cherry, par. 2–3). Why did people say that Phineas Gage was a “

In [12]:
tags = [tag for tag_list in df['reduced_tags'] for tag in tag_list ]
print(len(list(set(tags))))

30


In [13]:
train, test = train_test_split(df, test_size = TEST_SIZE, random_state= RANDOM_STATE)
    
train_token_list = list(train[TEXT_TOKEN_COL_NAME])
train_class_list = list(train[CLASS_COL_NAME])
test_token_list = list(test[TEXT_TOKEN_COL_NAME])
test_class_list = list(test[CLASS_COL_NAME])

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

enc = TfidfVectorizer(analyzer = lambda x:x, decode_error = 'ignore')


def class2onehot(lc, mode="train"):
    if mode=="train":
        lenc = CountVectorizer(analyzer = lambda x:x, decode_error='ignore')
        lenc.fit(lc)
    return lenc.transform(lc).toarray()

train_x_onehot = enc.fit_transform(train_token_list).toarray()
train_y_onehot = class2onehot(train_class_list)
test_x_onehot = enc.transform(test_token_list).toarray()
test_y_onehot = class2onehot(test_class_list)

In [15]:
train_y_onehot = train_y_onehot.T

# Logistic Chain

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
y_pred_list = list()
y_pred_prob_list = list()

for i in range(train_y_onehot.shape[0]):
  x_train = train_x_onehot
  x_test = test_x_onehot
  y_train = train_y_onehot[i]
  model = LogisticRegression()
  model.fit(x_train, y_train)
  y_pred = model.predict(x_test)
  y_pred_prob = model.predict_proba(x_test)
  y_pred_list.append(y_pred)
  y_pred_prob_list.append(y_pred_prob)
  print("have already deal with ", i+1 , "models")

have already deal with  1 models
have already deal with  2 models
have already deal with  3 models
have already deal with  4 models
have already deal with  5 models
have already deal with  6 models
have already deal with  7 models
have already deal with  8 models
have already deal with  9 models
have already deal with  10 models
have already deal with  11 models
have already deal with  12 models
have already deal with  13 models
have already deal with  14 models
have already deal with  15 models
have already deal with  16 models
have already deal with  17 models
have already deal with  18 models
have already deal with  19 models
have already deal with  20 models
have already deal with  21 models
have already deal with  22 models
have already deal with  23 models
have already deal with  27 models
have already deal with  28 models
have already deal with  29 models
have already deal with  30 models


In [34]:
y_pred_array = np.array(y_pred_list)
y_pred_array = y_pred_array.T

In [35]:
y_pred_prob_array = np.array(y_pred_prob_list)
y_pred_prob_array = y_pred_prob_array.T

In [36]:
from sklearn.preprocessing import MultiLabelBinarizer
MLB = MultiLabelBinarizer()
MLB.fit(df['reduced_tags'])  # array([[1, 1, 0...], [0, 1, 0...] ...])

labels = MLB.transform(test['reduced_tags'])

In [37]:
# Function: 計算 Accuracy, Precision, Recall, Top-k

def _ACCscore(y_true, pred):
    accuracy_l = [ans.all() for ans in (pred == y_true)]
    accuracy = np.array(accuracy_l).mean()
    return accuracy


def _PRscore(y_true, pred):
    hit_matrix = np.zeros_like(pred)
    hit_matrix[np.where((pred == y_true) & (y_true > 0))] = 1
    tp = hit_matrix.sum(axis=1)
    pred_sum = pred.sum(axis=1)
    true_sum = y_true.sum(axis=1)
    precision_l = []
    recall_l = []
    for ix in range(tp.shape[0]):
        precision_score = (1.0 if true_sum[ix] == 0 else 0.0) if pred_sum[ix] == 0 else tp[ix]/pred_sum[ix]
        recall_score = (1.0 if pred_sum[ix] == 0 else 0.0) if true_sum[ix] == 0 else tp[ix]/true_sum[ix]
        precision_l.append(precision_score)
        recall_l.append(recall_score)
    precision = np.array(precision_l).mean()
    recall = np.array(recall_l).mean()
    return precision, recall


def _TopKscore(y_true, pred, output, k):
    topk_class = np.argsort(output, axis=1)[:,-k:]
    topk_pred = torch.zeros_like(pred)
    for i in range(pred.shape[0]):
        if pred[i, :].sum() > 0:
            topk_pred[i, topk_class[i]] = 1
    topk_precision, topk_recall = _PRscore(y_true, topk_pred)
    return topk_precision, topk_recall

# Function: 一次同時計算所有評估指標

def score(y_true, pred, output, k=5):
    accuracy = _ACCscore(y_true, pred)
    precision, recall = _PRscore(y_true, pred)
    _, topk_recall = _TopKscore(y_true, pred, output, k)
    return accuracy, precision, recall, topk_recall

In [38]:
import torch

## 原本模型預測結果

In [39]:
A_score_list = []
P_score_list = []
R_score_list = []

for i in range(len(labels)):
    A_score_list.append(_ACCscore(torch.tensor([labels[i]]), torch.tensor([y_pred_array[i]])))
    precision, recall = _PRscore(torch.tensor([labels[i]]), torch.tensor([y_pred_array[i]]))
    P_score_list.append(precision)
    R_score_list.append(recall)
print(np.array(A_score_list).mean())
print(np.array(P_score_list).mean())
print(np.array(R_score_list).mean())

/home/adam/lalami/lalami_env/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  


0.09409492273730684
0.18837380452003438
0.16998713


In [40]:
import torch.nn as nn
sigmoid = nn.Sigmoid()

## 強迫至少有一個答案

In [41]:
new_predict_list = []
for i in range(len(y_pred_prob_array[0])):
    if(sum(y_pred_array[i])) > 0:
        new_predict = y_pred_array[i]
    else:
        new_predict = np.zeros_like(torch.tensor(y_pred_array[0]))
        idx = torch.argmax(sigmoid(torch.tensor(y_pred_prob_array[1][i]))).item() 
        new_predict[idx] = 1
        
    new_predict_list.append(new_predict)
new_predict_tensor = torch.tensor(new_predict_list)

In [42]:
new_predict_tensor

tensor([[0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [43]:
A_score_list = []
P_score_list = []
R_score_list = []

for i in range(len(labels)):
    A_score_list.append(_ACCscore(torch.tensor([labels[i]]), new_predict_tensor[i]))
    precision, recall = _PRscore(torch.tensor([labels[i]]), torch.tensor([new_predict_list[i]]))
    P_score_list.append(precision)
    R_score_list.append(recall)
print(np.array(A_score_list).mean())
print(np.array(P_score_list).mean())
print(np.array(R_score_list).mean())

0.31456953642384106
0.58600074
0.46779802


# DT Chain

In [ ]:
# 機器跑不動

In [16]:
from sklearn import tree

In [ ]:
y_pred_list = list()
y_pred_prob_list = list()

for i in range(train_y_onehot.shape[0]):
  x_train = train_x_onehot
  x_test = test_x_onehot
  y_train = train_y_onehot[i]
  model = tree.DecisionTreeClassifier()
  model.fit(x_train, y_train)
  y_pred = model.predict(x_test)
  y_pred_prob = model.predict_proba(x_test)
  y_pred_list.append(y_pred)
  y_pred_prob_list.append(y_pred_prob)
  print("have already deal with ", i+1 , "models")